In [1]:
import sys
import os
# Add the parent directory to the Python path

# Change the current working directory to /app
os.chdir('/app')
from dotenv import load_dotenv
load_dotenv(dotenv_path='config/.env')
database_path = os.getenv('DATABASE_PATH')
from utils.db.fetch import fetch_entries
print(f'DATABASE_PATH: {database_path}')
market_name = 'indian_equity'
timeframe='1d'

DATABASE_PATH: database/db/ohlcv_data.db


In [38]:
import cloudscraper
import time

def fetch_sol_addresses_with_cloudscraper():
    url = "https://app.geckoterminal.com/api/p1/solana/pools"
    params = {
        "include": "dex,dex.network,dex.network.network_metric,tokens",
        "page": 1,
        "include_network_metrics": "true",
        "include_meta": "1",
        "volume_24h[gte]": "1000000",
        "pool_creation_hours_ago[lte]": "1200",
        "has_social": "1",
        "sort": "-24h_volume",
        "networks": "solana",
    }
    headers = {
        "User-Agent": "PostmanRuntime/7.42.0",
        "Accept": "*/*",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
    }

    scraper = cloudscraper.create_scraper()  # Create a scraper object that bypasses Cloudflare
    scraper.headers.update(headers)  # Update headers for all requests
    
    sol_addresses = []

    while True:
        response = scraper.get(url, params=params)

        if response.status_code == 403:
            print("403 Forbidden: Ensure you are authorized to access the API.")
            break
        elif response.status_code != 200:
            print(f"Error: Unable to fetch data, status code {response.status_code}")
            break

        print(response.text)
        data = response.json()
        print(data)
        pools = data.get("data", [])
        
        for pool in pools:
            attributes = pool.get("attributes", {})
            name = attributes.get("name", "")
            address = attributes.get("address", "")
            if name.lower().endswith("/ sol"):
                sol_addresses.append(address)

        links = data.get("links", {})
        next_page = links.get("next")
        if not next_page:
            break

        params["page"] += 1
        time.sleep(3)

    return sol_addresses



In [39]:

addresses = fetch_sol_addresses_with_cloudscraper()
print(f"Found {len(addresses)} addresses ending with '/ SOL':")
print(addresses)

{"data":[{"id":"168691032","type":"pool","attributes":{"address":"93tjgwff5Ac5ThyMi8C4WejVVQq4tuMeMuYW1LEYZ7bu","name":"CHILLGUY / SOL","from_volume_in_usd":"235101715.330769","to_volume_in_usd":"235101715.330769","api_address":"93tjgwff5Ac5ThyMi8C4WejVVQq4tuMeMuYW1LEYZ7bu","swap_count_24h":214145,"price_percent_change":"+740.29%","price_percent_changes":{"last_5m":"+0.99%","last_15m":"-3.64%","last_30m":"-11.79%","last_1h":"+12.64%","last_6h":"+20.59%","last_24h":"+740.31%"},"pool_fee":null,"base_token_id":"36213287","token_value_data":{"4045901":{"fdv_in_usd":140595067218.45065,"market_cap_in_usd":0.0,"market_cap_to_holders_ratio":175052.20153902512},"36213287":{"fdv_in_usd":247357460.95095995,"market_cap_in_usd":247357461.99937516,"market_cap_to_holders_ratio":4281.868018751501}},"price_in_usd":"0.247369234724918","reserve_in_usd":"3366721.07202","pool_created_at":"2024-11-15T20:08:23.000Z","aggregated_network_metrics":{"total_swap_volume_usd_24h":"17674352897.2117558620488373","tot

In [32]:
import cloudscraper

url = "https://app.geckoterminal.com/api/p1/solana/pools?include=dex%2Cdex.network%2Cdex.network.network_metric%2Ctokens&page=4&include_network_metrics=true&include_meta=1&volume_24h%5Bgte%5D=1000000&pool_creation_hours_ago%5Blte%5D=1200&has_social=1&sort=-24h_volume&networks=solana"

headers = {
    "User-Agent": "PostmanRuntime/7.42.0",
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
}

scraper = cloudscraper.create_scraper()  # Handles Cloudflare and Brotli encoding
response = scraper.get(url, headers=headers)

if response.status_code == 200:
    print(response.text)  # Decoded response content
else:
    print(f"Failed with status code: {response.status_code}")


{"data":[{"id":"168719510","type":"pool","attributes":{"address":"DiRTSpizmJfqNKUsCiQPxacwppYV8BvURVERTQifQUgA","name":"scihub / SOL","from_volume_in_usd":"1806510.880466","to_volume_in_usd":"1806510.880466","api_address":"DiRTSpizmJfqNKUsCiQPxacwppYV8BvURVERTQifQUgA","swap_count_24h":2683,"price_percent_change":"+51.98%","price_percent_changes":{"last_5m":"+2.44%","last_15m":"+7.15%","last_30m":"+11.43%","last_1h":"+13.37%","last_6h":"+132.84%","last_24h":"+51.98%"},"pool_fee":null,"base_token_id":"36221254","token_value_data":{"4045901":{"fdv_in_usd":140271291784.16623,"market_cap_in_usd":0.0,"market_cap_to_holders_ratio":174750.20653418757},"36221254":{"fdv_in_usd":40562601.22213444,"market_cap_in_usd":0.0,"market_cap_to_holders_ratio":2739.5011971888252}},"price_in_usd":"0.0408993202492667","reserve_in_usd":"303676.92122","pool_created_at":"2024-11-16T13:28:25.000Z","aggregated_network_metrics":{"total_swap_volume_usd_24h":"17712210376.1868450757900473","total_swap_volume_usd_48h_2

In [41]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def fetch_ohlcv_data(address, days, resolution="minute", aggregate=5):
    """
    Fetch OHLCV data for a given address over the last `days` with specified resolution and aggregate.

    Parameters:
        address (str): The pool address (e.g., from earlier API call).
        days (int): Number of days of data to fetch.
        resolution (str): Timeframe resolution ("minute", "hour", or "day").
        aggregate (int): Aggregation period (default: 5).
    
    Returns:
        pandas.DataFrame: OHLCV data with columns ['timestamp', 'open', 'high', 'low', 'close', 'volume'].
    """
    base_url = f"https://api.geckoterminal.com/api/v2/networks/solana/pools/{address}/ohlcv/{resolution}"
    headers = {
        "User-Agent": "PostmanRuntime/7.42.0",
        "Accept": "application/json",
    }
    params = {
        "aggregate": aggregate,
        "limit": 1000,  # Max limit per API call
        "currency": "USD",
        "token": "base",
    }

    # Initialize variables for pagination
    current_timestamp = int(datetime.utcnow().timestamp())  # Current timestamp in epoch
    start_timestamp = int((datetime.utcnow() - timedelta(days=days)).timestamp())
    all_data = []

    while current_timestamp > start_timestamp:
        # Update the `before_timestamp` parameter
        params["before_timestamp"] = current_timestamp

        # Make the API request
        response = requests.get(base_url, headers=headers, params=params)
        if response.status_code != 200:
            print(f"Failed to fetch data: {response.status_code} - {response.text}")
            break

        # Parse the response
        json_data = response.json()
        ohlcv_data = json_data.get("data", {}).get("attributes", {}).get("ohlcv_list", [])
        if not ohlcv_data:  # Break if no more data
            break

        # Append data to the list
        all_data.extend(ohlcv_data)

        # Update `current_timestamp` for pagination (use the earliest timestamp from the fetched data)
        current_timestamp = ohlcv_data[-1][0]

    # Convert to a DataFrame
    data = pd.DataFrame(all_data, columns=["timestamp", "open", "high", "low", "close", "volume"])

    # Format the timestamp column
    if not data.empty:
        data["timestamp"] = pd.to_datetime(data["timestamp"], unit="s")
        data["timestamp"] = data["timestamp"].dt.strftime('%Y-%m-%d %H:%M:%S')

    return data.sort_values("timestamp")  # Return sorted data


In [42]:
pool_address = "HobaNfHNrDcFP8z8T9n3j6K4UF5jSDmdpdxXQpyC83o"
days = 15
resolution = "minute"
aggregate = 5

ohlcv_data = fetch_ohlcv_data(pool_address, days, resolution, aggregate)
print(ohlcv_data)


                timestamp      open      high       low     close  \
1866  2024-11-13 21:30:00  0.000073  0.000099  0.000060  0.000064   
1865  2024-11-13 21:35:00  0.000064  0.000347  0.000060  0.000189   
1864  2024-11-13 21:40:00  0.000189  0.000245  0.000153  0.000194   
1863  2024-11-13 21:45:00  0.000194  0.000205  0.000135  0.000136   
1862  2024-11-13 21:50:00  0.000136  0.000136  0.000088  0.000088   
...                   ...       ...       ...       ...       ...   
4     2024-11-20 08:40:00  0.152019  0.153498  0.143542  0.145759   
3     2024-11-20 08:45:00  0.145759  0.146357  0.143529  0.146357   
2     2024-11-20 08:50:00  0.146357  0.148888  0.143940  0.147627   
1     2024-11-20 08:55:00  0.147627  0.148643  0.144031  0.146203   
0     2024-11-20 09:00:00  0.146203  0.146935  0.146070  0.146070   

             volume  
1866   24807.832231  
1865  199536.109302  
1864   16675.233101  
1863    6501.743108  
1862    9196.430354  
...             ...  
4     102439.1333

In [43]:
ohlcv_data

,timestamp,open,high,low,close,volume
1866,2024-11-13 21:30:00,0.000073,0.000099,0.000060,0.000064,24807.832231
1865,2024-11-13 21:35:00,0.000064,0.000347,0.000060,0.000189,199536.109302
1864,2024-11-13 21:40:00,0.000189,0.000245,0.000153,0.000194,16675.233101
1863,2024-11-13 21:45:00,0.000194,0.000205,0.000135,0.000136,6501.743108
1862,2024-11-13 21:50:00,0.000136,0.000136,0.000088,0.000088,9196.430354
...,...,...,...,...,...,...
4,2024-11-20 08:40:00,0.152019,0.153498,0.143542,0.145759,102439.133316
3,2024-11-20 08:45:00,0.145759,0.146357,0.143529,0.146357,28805.540569
2,2024-11-20 08:50:00,0.146357,0.148888,0.143940,0.147627,53323.070753
1,2024-11-20 08:55:00,0.147627,0.148643,0.144031,0.146203,69352.257712
